# NVCL Spectral Data Download Notebook

This notebook demonstrates how you can query the NVCLDataservices to download raw spectral reflectance data to csv files.

Set the 'state' variable to select the state you want to query from.  Then run the notebook.

The notebook will loop over the first 'maxboreholes' datasets available from the selected state and save the Reflectance spectral data into csv files named 'state/datasetname_spectra.csv'.  Adjust the 'maxboreholes' variable as high as you like but be aware that increasing this value will cause a proportional increase in execution time and disk space consumption.  For example, 20 boreholes from SA takes me around 15 minutes and consumes around 5Gb of disk space.

the 'spectrallayername' variable can also be changed to 'Base Refl' to get thermal spectra. Note: not all datasets will have thermal.

In [21]:
from nvcl_kit.reader import NVCLReader
from nvcl_kit.param_builder import param_builder
import array
import os
import numpy as np

# state can be:  'csiro', 'nsw', 'tas', 'vic', 'qld', 'sa', 'wa', 'nt'
state = 'nsw'

# change spectrallayername to 'Base Refl' for thermal data. Note: not all datasets will have thermal.
spectrallayername = 'Reflectance'

maxboreholes = 20


In [22]:
param = param_builder(state, max_boreholes=maxboreholes)
if not param:
    print(f"Cannot build parameters: {param}")


# Initialise reader
reader = NVCLReader(param)
if not reader.wfs:
    print(f"ERROR! Cannot contact WFS service for {state}")


In [23]:
nvcl_id_list = reader.get_nvcl_id_list()


In [24]:


for nvcl_id in nvcl_id_list[0:maxboreholes]:
    spectrallog_data_list = reader.get_spectrallog_data(nvcl_id)
    isExist = os.path.exists(state)
    if not isExist:
        os.makedirs(state)
    for sld in spectrallog_data_list:
        if (sld.log_name == spectrallayername):
            sl_bin = reader.get_spectrallog_datasets(sld.log_id, start_sample_no="0", end_sample_no=sld.sample_count-1)
            if len(sl_bin) != sld.sample_count * len(sld.wavelengths) * 4:
                print(f"ERROR! {nvcl_id}  expected {sld.sample_count * len(sld.wavelengths) * 4} bytes but got {len(sl_bin)}")
                continue
            f = open(state +"\\" + nvcl_id +'_' + spectrallayername+"_spectra.csv", "w")
            f.write(','.join(str(item) for item in sld.wavelengths)+'\n')

            wvlcount = len(sld.wavelengths)
            for i in range(sld.sample_count):
                U = array.array("f")
                U.frombytes(sl_bin[(i*wvlcount)*4:((i+1)*wvlcount)*4])
                f.write(','.join(str(item) for item in U)+'\n')

            f.close()


nvcl_kit.svc_interface -- WARNING - _get_response_str: OS Error: HTTP Error 500: 


ERROR! COAL_070748  expected 66164912 bytes but got 0
nvcl_kit.svc_interface -- WARNING - _get_response_str: OS Error: HTTP Error 500: 


ERROR! MIN_000180  expected 39756508 bytes but got 0
nvcl_kit.svc_interface -- WARNING - _get_response_str: OS Error: HTTP Error 500: 


ERROR! MIN_000539  expected 9752600 bytes but got 0
